In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/associative-rules-data/retail_transactional.csv
/kaggle/input/associative-rules-data/groceries.csv


In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import csv
import time
import tracemalloc
from itertools import combinations
from collections import defaultdict


In [4]:
import os
from collections import Counter

# File paths
GROCERIES_PATH = '/kaggle/input/associative-rules-data/groceries.csv'
RETAIL_PATH = '/kaggle/input/associative-rules-data/retail_transactional.csv'

def inspect_and_analyze(file_path, name):
    print(f"\n{'='*50}")
    print(f" Analyzing Dataset: {name}")
    print(f"{'='*50}")
    
    if not os.path.exists(file_path):
        print(f" File not found: {file_path}")
        return

    # 1. Preview raw content (First 5 lines) to understand structure
    print(" Raw File Content (First 5 lines):")
    try:
        with open(file_path, 'r', encoding='utf-8', errors='replace') as f:
            raw_lines = [f.readline().strip() for _ in range(5)]
            for i, line in enumerate(raw_lines):
                print(f"  Line {i+1}: {line}")
    except Exception as e:
        print(f"  Could not read raw lines: {e}")
        return

    # 2. Process file line by line to calculate statistics
    transactions = []
    unique_items = set()
    
    try:
        with open(file_path, 'r', encoding='utf-8', errors='replace') as f:
            for line in f:
                line = line.strip()
                if not line: continue
                
                # Split by comma (assuming standard CSV or basket format)
                # We remove quotes to clean up data like "milk" -> milk
                items = [i.strip().replace('"', '') for i in line.split(',')]
                
                # Filter out empty strings
                items = [i for i in items if i]
                
                if items:
                    transactions.append(items)
                    unique_items.update(items)

        # 3. Calculate Statistics
        num_transactions = len(transactions)
        num_unique_items = len(unique_items)
        
        if num_transactions == 0:
            print(" Dataset appears empty.")
            return

        avg_len = sum(len(t) for t in transactions) / num_transactions
        
        # Calculate Top 5 Frequent Items
        all_items = [item for t in transactions for item in t]
        item_counts = Counter(all_items)
        top_5 = item_counts.most_common(5)

        print(f"\n Statistics Summary:")
        print(f"  • Total Transactions (Rows): {num_transactions:,}")
        print(f"  • Unique Items Found: {num_unique_items:,}")
        print(f"  • Avg Items per Row: {avg_len:.2f}")
        
        print(f"\n Top 5 Frequent Items (Check for headers or strange IDs):")
        for item, count in top_5:
            support = count / num_transactions
            print(f"  - '{item}': {count} times (Freq: {support:.4f})")

    except Excep as e:
        print(f" Error calculating stats: {e}")

# Run analysis
inspect_and_analyze(GROCERIES_PATH, "Groceries")
inspect_and_analyze(RETAIL_PATH, "Retail Transactional")



 Analyzing Dataset: Groceries
 Raw File Content (First 5 lines):
  Line 1: citrus fruit,semi-finished bread,margarine,ready soups
  Line 2: tropical fruit,yogurt,coffee
  Line 3: whole milk
  Line 4: pip fruit,yogurt,cream cheese ,meat spreads
  Line 5: other vegetables,whole milk,condensed milk,long life bakery product

 Statistics Summary:
  • Total Transactions (Rows): 9,835
  • Unique Items Found: 169
  • Avg Items per Row: 4.41

 Top 5 Frequent Items (Check for headers or strange IDs):
  - 'whole milk': 2513 times (Freq: 0.2555)
  - 'other vegetables': 1903 times (Freq: 0.1935)
  - 'rolls/buns': 1809 times (Freq: 0.1839)
  - 'soda': 1715 times (Freq: 0.1744)
  - 'yogurt': 1372 times (Freq: 0.1395)

 Analyzing Dataset: Retail Transactional
 Raw File Content (First 5 lines):
  Line 1: 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
  Line 2: 30,31,32
  Line 3: 33,34,35
  Line 4: 36,37,38,39,40,41,42,43,44,45,46
  Line 5: 38,39,47,48

 Statistics Summ

# Analysis of Groceries Dataset (The “Control” Group)
- Structure: It is a Sparse Dataset.
- Volume: ~10k transactions.
- Dimensionality: Only 169 unique items.
- Implication for Algorithms:
- Standard Apriori: Will run very fast. Generating pairs from 169 items is trivial for modern CPUs.
- DHP:  probably won’t see a huge performance gain here because the overhead of hashing might outweigh the benefit of pruning such a small candidate set.
- Sampling: With 10k rows, a 10% sample is only ~980 rows. This is small enough that statistical variance might cause some False Negatives at low support thresholds.

# Analysis of Retail Transactional Dataset (The “Stress Test”)
- Structure: It is a High-Dimensional / Dense Core Dataset.
- Volume: ~88k transactions (9x larger than Groceries).
- Dimensionality: 16,470 Unique Items!
- The “Explosion” Risk:
- In Standard Apriori, step k= 2  tries to generate pairs from frequent single items.
- Worst case scenario (if many items are frequent):135,630,465 pairs.
- Item Distribution: Item '39' appears in 57% of baskets. Item '48' in 47%. This means L1 (Frequent 1-itemsets) will be large, guaranteeing that C2 will be huge.


In [5]:
# ==========================================
# 1. DATA LOADING UTILS
# ==========================================
def load_dataset(file_path):
    
    dataset = []
    print(f" Loading: {file_path} ...")
    try:
        with open(file_path, 'r', encoding='utf-8', errors='replace') as f:
            for line in f:
                line = line.strip()
                if not line: continue
                # Clean and split items
                items = [i.strip().replace('"', '') for i in line.split(',')]
                items = [i for i in items if i] # Remove empty strings
                if items:
                    dataset.append(frozenset(items))
        print(f" Loaded {len(dataset)} transactions.")
        return dataset
    except Exception as e:
        print(f" Error loading file: {e}")
        return []

In [6]:
# ==========================================
# 2. HELPER FUNCTIONS (The Engine)
# ==========================================

def create_c1(dataset):
    c1 = {}
    for transaction in dataset:
        for item in transaction:
            itemset = frozenset([item])
            c1[itemset] = c1.get(itemset, 0) + 1
    return c1

def filter_candidates(candidates_counts, min_support_count):
    ret_l = {}
    for key, value in candidates_counts.items():
        if value >= min_support_count:
            ret_l[key] = value
    return ret_l

def generate_Ck(Lk_minus_1, k):
   
    candidates = set()
    lk_list = list(Lk_minus_1.keys())
    len_lk = len(lk_list)
    
    # Join Step
    for i in range(len_lk):
        for j in range(i + 1, len_lk):
            l1 = list(lk_list[i])
            l2 = list(lk_list[j])
            l1.sort()
            l2.sort()
            
            # Check if first k-2 items are equal
            if l1[:k-2] == l2[:k-2]:
                # Union creates the new candidate
                candidate = lk_list[i] | lk_list[j]
                
                # Prune Step: Check if all subsets are frequent
                # (Optional for speed in Python, but standard for strict Apriori)
                if has_infrequent_subset(candidate, Lk_minus_1):
                    continue
                candidates.add(candidate)
    return candidates

def has_infrequent_subset(candidate, Lk_minus_1):
    # generate all subsets of size k-1
    for subset in combinations(candidate, len(candidate) - 1):
        if frozenset(subset) not in Lk_minus_1:
            return True
    return False

def count_candidates(dataset, candidates):
    counts = {cand: 0 for cand in candidates}
    
    # Optimization: Iterate through dataset once
    for transaction in dataset:
        # For each candidate, check if it is in transaction
        # Optimization for k=2: Double loop over transaction items is sometimes faster than subset check
        # But for general k, issubset is safest in Python
        for cand in candidates:
            if cand.issubset(transaction):
                counts[cand] += 1
    return counts


In [7]:
# ==========================================
# 3. STANDARD APRIORI ALGORITHM (Instrumented)
# ==========================================

def run_standard_apriori(dataset, min_support_fraction):
    
    tracemalloc.start()
    start_time = time.time()
    
    min_support_count = int(len(dataset) * min_support_fraction)
    stats = {
        "algorithm": "Standard Apriori",
        "min_sup": min_support_fraction,
        "c2_count": 0, # To store |C2|
        "total_freq_itemsets": 0,
        "execution_time": 0,
        "memory_peak_mb": 0
    }
    
    L_global = {}
    
    # --- Step 1: C1 & L1 ---
    print(f"   [Standard] Generating L1 (Min Count: {min_support_count})...")
    C1_counts = create_c1(dataset)
    L1 = filter_candidates(C1_counts, min_support_count)
    L_global.update(L1)
    
    L_current = L1
    k = 2
    
    while L_current:
        # print(f"   [Standard] Step k={k}...")
        
        # Generate Candidates
        Ck = generate_Ck(L_current, k)
        
        # *** CAPTURE |C2| METRIC ***
        if k == 2:
            stats["c2_count"] = len(Ck)
            print(f"   [Metric Captured] |C2_standard| = {len(Ck)}")
        
        if not Ck:
            break
            
        # Count Candidates
        Ck_counts = count_candidates(dataset, Ck)
        
        # Filter
        Lk = filter_candidates(Ck_counts, min_support_count)
        
        if not Lk:
            break
            
        L_global.update(Lk)
        L_current = Lk
        k += 1

    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    
    stats["total_freq_itemsets"] = len(L_global)
    stats["execution_time"] = end_time - start_time
    stats["memory_peak_mb"] = peak / (1024 * 1024)
    
    return L_global, stats

In [8]:


datasets_config = [
    {
        'name': 'GROCERIES',
        'path': '/kaggle/input/associative-rules-data/groceries.csv'
    },
    {
        'name': 'RETAIL',
        'path': '/kaggle/input/associative-rules-data/retail_transactional.csv' 
    }
]

# The list of support values to test
min_sup_list = [0.05, 0.03, 0.02, 0.01]

# Container for all results
all_results = []

# ==========================================
# EXECUTION LOOP
# ==========================================

for ds in datasets_config:
    print(f"\n{'#'*60}")
    print(f" PROCESSING DATASET: {ds['name']}")
    print(f"{'#'*60}")
    
    # 1. Load the dataset
    dataset = load_dataset(ds['path'])
    
    if not dataset:
        print(f" Failed to load {ds['name']}. Skipping...")
        continue

    # 2. Run the loop for each min_sup
    print(f"   Dataset size: {len(dataset)} transactions.")
    
    for min_sup in min_sup_list:
        print(f"\n    Testing Min Support: {min_sup}")
        
        try:
            # Run Standard Apriori
            _, metrics = run_standard_apriori(dataset, min_support_fraction=min_sup)
            
            # Tag the metrics with the dataset name for the final table
            metrics['dataset'] = ds['name']
            all_results.append(metrics)
            
            print(f"       Done.")
            print(f"      • |C2_standard|:    {metrics['c2_count']}")
            print(f"      • Frequent Sets:    {metrics['total_freq_itemsets']}")
            print(f"      • Execution Time:   {metrics['execution_time']:.4f} s")
            print(f"      • Peak Memory:      {metrics['memory_peak_mb']:.4f} MB")
            
        except Exception as e:
            print(f"       Error running {ds['name']} at {min_sup}: {e}")

# ==========================================
# FINAL COMPARISON TABLE
# ==========================================
print(f"\n{'='*60}")
print(" FINAL COMBINED SUMMARY (Standard Apriori)")
print(f"{'='*60}")

if all_results:
    df_results = pd.DataFrame(all_results)
    # Reorder columns: Dataset first, then the metrics
    cols = ['dataset', 'min_sup', 'c2_count', 'total_freq_itemsets', 'execution_time', 'memory_peak_mb']
    print(df_results[cols].to_string(index=False))
else:
    print("No results to display.")



############################################################
 PROCESSING DATASET: GROCERIES
############################################################
 Loading: /kaggle/input/associative-rules-data/groceries.csv ...
 Loaded 9835 transactions.
   Dataset size: 9835 transactions.

    Testing Min Support: 0.05
   [Standard] Generating L1 (Min Count: 491)...
   [Metric Captured] |C2_standard| = 378
       Done.
      • |C2_standard|:    378
      • Frequent Sets:    31
      • Execution Time:   0.4253 s
      • Peak Memory:      0.1853 MB

    Testing Min Support: 0.03
   [Standard] Generating L1 (Min Count: 295)...
   [Metric Captured] |C2_standard| = 946
       Done.
      • |C2_standard|:    946
      • Frequent Sets:    64
      • Execution Time:   0.8476 s
      • Peak Memory:      0.3253 MB

    Testing Min Support: 0.02
   [Standard] Generating L1 (Min Count: 196)...
   [Metric Captured] |C2_standard| = 1711
       Done.
      • |C2_standard|:    1711
      • Frequent Sets:    1

### 📊 Analysis of Standard Apriori Baseline (Groceries Dataset)

The execution of the Standard Apriori algorithm on the `Groceries` dataset establishes a critical baseline. By varying the `min_support` from 0.05 down to 0.01, we can clearly observe the computational cost associated with the **Combinatorial Explosion** inherent in the Apriori principle.

#### 1. The Non-Linear Growth of Candidates ($|C_2|$)
The most significant insight from this test is the behavior of $|C_{2\_standard}|$ (the count of 2-itemset candidates).
*   **Observation:** When `min_sup` was lowered from **0.05** to **0.01** (a 5x reduction in threshold), the number of candidates generated did not increase linearly. Instead, it jumped from **378** to **3,828** (a **~10x increase**).
*   **Insight:** This illustrates the quadratic nature of candidate generation. As the support threshold drops, slightly more individual items ($L_1$) become "frequent." Since $C_2$ is formed by combining all members of $L_1$ (computationally $\binom{n}{2}$), even a small increase in $L_1$ size results in a massive spike in potential pairs that the algorithm must verify against the database.

#### 2. Performance Scaling (Time & Memory)
*   **Execution Time:** The runtime increased from **0.43s** to **3.79s**. While ~4 seconds is manageable for this specific dataset, the growth curve is steep. This confirms that Standard Apriori is highly sensitive to the support threshold.
*   **Memory Usage:** Peak memory usage grew from **0.18 MB** to **1.28 MB**. While the Groceries dataset is sparse enough that memory is not yet a hardware constraint, the upward trend confirms that memory consumption scales directly with the number of candidates stored in the hash tree/dictionary.

#### 3. Implications for Larger Datasets (Retail)
This baseline highlights the risk for the **Retail Transactional** dataset.
*   In `Groceries`, we have ~169 unique items, resulting in ~3,800 candidates at 1% support.
*   In `Retail`, with **16,470 unique items**, a similar Standard Apriori run could theoretically generate **millions** of candidates in $C_2$.
*   The 3.79s runtime observed here would likely translate to hours or reduced system stability on the larger dataset without optimization.

#### 4. Conclusion and Objective for DHP
We have successfully captured the target metric to beat: **$|C_{2\_standard}| = 3,828$** (at 1% support).

The success of the next algorithm, **DHP (Direct Hashing and Pruning)**, will be measured not necessarily by raw execution time (due to Python overhead), but by its ability to significantly reduce this candidate count. If DHP is effective, $|C_{2\_DHP}|$ should be substantially lower than 3,828, proving that we successfully pruned false positives before scanning the database.


In [9]:

# DHP HELPER FUNCTIONS

def get_hash_bucket_index(item1, item2, bucket_count):
    # hash() in Python is consistent within a session
    return (hash(item1) ^ hash(item2)) % bucket_count

def scan_and_count_supports(dataset, candidates):
    """
    Standard scan to count supports for candidates.
    """
    counts = defaultdict(int)
    
    candidate_set = set(candidates)
    
    for transaction in dataset:
        # Convert transaction to set for fast subset checking
        t_set = frozenset(transaction)
        
        # Check which candidates are in this transaction
        # (For very large candidate sets, a Hash Tree is better, 
        # but for this assignment, simple subset check is fine)
        for cand in candidates:
            if cand.issubset(t_set):
                counts[cand] += 1
    return counts

def generate_next_candidates_standard(L_k_minus_1, k):
    
    candidates = []
    l_list = sorted(list(L_k_minus_1), key=lambda x: list(x))
    n = len(l_list)
    
    for i in range(n):
        for j in range(i + 1, n):
            itemset1 = list(l_list[i])
            itemset2 = list(l_list[j])
            
            # Join condition: first k-2 items must be identical
            # For k=3, check first 1 item.
            if k == 2 or itemset1[:k-2] == itemset2[:k-2]:
                # Union
                new_cand = l_list[i] | l_list[j]
                # Pruning Step (Apriori Property):
                candidates.append(new_cand)
                
    return candidates


# MAIN DHP ALGORITHM

def run_dhp_apriori(dataset, min_support_fraction, bucket_count=1000003):
    
    # 1Setup & Metrics
    tracemalloc.start()
    start_time = time.time()
    
    total_transactions = len(dataset)
    min_count = total_transactions * min_support_fraction
    
    global_freq_itemsets = {}
    
    print(f"   [DHP] Started. Min Sup Count: {min_count:.1f}, Buckets: {bucket_count}")

    #  Init Pass (Count L1 + Populate Hash Table)

    print("   [DHP] Pass 1: Counting Items & Hashing Pairs...")
    
    item_counts = defaultdict(int)
    bucket_table = defaultdict(int) # Using dict instead of list to save memory on sparse buckets
    
    for transaction in dataset:
        items = list(transaction) # Convert to list for indexing
        
        # 1. Count individual items
        for item in items:
            item_counts[item] += 1
            
        # We do this loop HERE to avoid reading database again for C2 generation logic
        n_items = len(items)
        if n_items >= 2:
            for i in range(n_items):
                for j in range(i + 1, n_items):
                    b_idx = get_hash_bucket_index(items[i], items[j], bucket_count)
                    bucket_table[b_idx] += 1

    # Generate L1
    L1 = [frozenset([item]) for item, count in item_counts.items() if count >= min_count]
    
    # Store L1 counts
    for l1 in L1:
        global_freq_itemsets[l1] = item_counts[list(l1)[0]]
        
    print(f"   [DHP] L1 Size: {len(L1)}")

    # Generate C2 using DHP Pruning
    
    # Standard Apriori would just join L1.
    # DHP checks the bucket_table before adding to C2.
    
    C2_candidates = []
    L1_single_items = [list(x)[0] for x in L1] # Unpack frozensets to items
    L1_single_items.sort() # Sorting helps determiistic behavior
    
    # Generate all pairs from L1
    potential_pairs = list(combinations(L1_single_items, 2))
    
    for p in potential_pairs:
        # Check Hash Table
        b_idx = get_hash_bucket_index(p[0], p[1], bucket_count)
        
        # PRUNING CONDITION:
        # If the bucket count is less than min_count, this pair CANNOT be frequent.
        if bucket_table[b_idx] >= min_count:
            C2_candidates.append(frozenset(p))
            
    # !!! CRITICAL METRIC !!!
    c2_count = len(C2_candidates)
    print(f"   [Metric Captured] |C2_DHP| = {c2_count} (Pruned from {len(potential_pairs)} possible)")

    # Standard Loop (Scan for C2, Generate L2, C3...)
    current_candidates = C2_candidates
    k = 2
    
    while current_candidates:
        
        # 1. Count Support for Current Candidates
        candidates_counts = scan_and_count_supports(dataset, current_candidates)
        
        # 2. Filter to get L_k
        L_k = []
        for cand, count in candidates_counts.items():
            if count >= min_count:
                L_k.append(cand)
                global_freq_itemsets[cand] = count
        
        
        # Stop if no frequent itemsets found
        if not L_k:
            break
            
        # Generate Next Candidates (C_k+1)
        k += 1
        current_candidates = generate_next_candidates_standard(L_k, k)
        
        # Safety break for very deep trees
        if k > 5: 
            break

    # FINISH & METRICS
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    
    execution_time_sec = end_time - start_time
    peak_memory_mb = peak / (1024 * 1024)
    
    return global_freq_itemsets, {
        'c2_count': c2_count,
        'total_freq_itemsets': len(global_freq_itemsets),
        'execution_time': execution_time_sec,
        'memory_peak_mb': peak_memory_mb
    }


In [14]:
import pandas as pd
from IPython.display import display

# ... (Loading Dataset Code remains the same) ...

min_supports = [0.05, 0.03, 0.02, 0.01]
results_dhp = []

print("Running DHP Tests...")

for name, data in datasets.items():
    if not data: continue 
    
    for min_sup in min_supports:
        # Run DHP
        freq_items, metrics = run_dhp_apriori(data, min_support_fraction=min_sup, bucket_count=1000003)
        
        # Save Results
        result_row = {
            "Dataset": name,
            "Min Support": min_sup,
            "|C2| (DHP Candidates)": metrics['c2_count'],
            "Total Freq Items": metrics['total_freq_itemsets'],
            "Time (s)": round(metrics['execution_time'], 4),
            "Memory (MB)": round(metrics['memory_peak_mb'], 4)
        }
        results_dhp.append(result_row)
        
        # Simple progress print (just dots or basic info to reduce clutter)
        print(f"   -> Finished {name} @ {min_sup} | Time: {metrics['execution_time']:.2f}s")

print("\n" + "="*30)
print("   ✅ TEST COMPLETED")
print("="*30 + "\n")

# --- BEAUTIFUL OUTPUT SECTION ---

# 1. Create DataFrame
df_results = pd.DataFrame(results_dhp)

# 2. Apply Styling (Heatmap)
# High values in Time and C2 will be Red, Low values will be Blue/White
styled_df = df_results.style.background_gradient(
    subset=['|C2| (DHP Candidates)', 'Time (s)', 'Memory (MB)'], 
    cmap='OrRd'  # Orange to Red colormap
).format({
    'Min Support': '{:.2f}',
    'Time (s)': '{:.4f}',
    'Memory (MB)': '{:.4f}'
}).set_properties(**{
    'text-align': 'center',
    'border': '1px solid black'
}).set_caption("🚀 DHP Apriori Performance Results")

# 3. Display
display(styled_df)


Running DHP Tests...
   [DHP] Started. Min Sup Count: 491.8, Buckets: 1000003
   [DHP] Pass 1: Counting Items & Hashing Pairs...
   [DHP] L1 Size: 28
   [Metric Captured] |C2_DHP| = 3 (Pruned from 378 possible)
   -> Finished GROCERIES @ 0.05 | Time: 0.80s
   [DHP] Started. Min Sup Count: 295.1, Buckets: 1000003
   [DHP] Pass 1: Counting Items & Hashing Pairs...
   [DHP] L1 Size: 44
   [Metric Captured] |C2_DHP| = 19 (Pruned from 946 possible)
   -> Finished GROCERIES @ 0.03 | Time: 0.82s
   [DHP] Started. Min Sup Count: 196.7, Buckets: 1000003
   [DHP] Pass 1: Counting Items & Hashing Pairs...
   [DHP] L1 Size: 59
   [Metric Captured] |C2_DHP| = 61 (Pruned from 1711 possible)
   -> Finished GROCERIES @ 0.02 | Time: 0.94s
   [DHP] Started. Min Sup Count: 98.4, Buckets: 1000003
   [DHP] Pass 1: Counting Items & Hashing Pairs...
   [DHP] L1 Size: 88
   [Metric Captured] |C2_DHP| = 213 (Pruned from 3828 possible)
   -> Finished GROCERIES @ 0.01 | Time: 1.64s
   [DHP] Started. Min Sup Coun

,Dataset,Min Support,|C2| (DHP Candidates),Total Freq Items,Time (s),Memory (MB)
0,GROCERIES,0.05,3,31,0.8030,81.7294
1,GROCERIES,0.03,19,63,0.8243,0.6118
2,GROCERIES,0.02,61,122,0.9441,0.6754
3,GROCERIES,0.01,213,333,1.6443,1.1296
4,RETAIL,0.05,7,16,35.3791,81.7293
5,RETAIL,0.03,16,32,35.5385,81.7291
6,RETAIL,0.02,23,55,35.5301,81.7291
7,RETAIL,0.01,58,159,35.8841,81.7292


# Phase 2 Analysis: DHP Apriori Results

Following the execution of the Direct Hashing and Pruning (DHP) algorithm, we can now compare its performance against the Standard Apriori baseline established in Phase 1. The results highlight a classic algorithmic trade-off between **search space reduction** and **computational overhead**.

### 1. The Major Victory: Massive Pruning ($|C_2|$ Reduction)
The primary goal of DHP is to reduce the size of the candidate set ($C_2$) generated in the second pass. The results show the algorithm succeeded spectacularly in this regard.

*   **Groceries Dataset (@ 0.01 Support):**
    *   **Standard Apriori:** Generated **3,828** candidates.
    *   **DHP Apriori:** Generated only **213** candidates.
    *   **Impact:** A **94.4% reduction** in the search space. This massive pruning resulted in the total execution time dropping from **3.79s** (Standard) to **1.64s** (DHP).

*   **Retail Dataset (@ 0.01 Support):**
    *   **Standard Apriori:** Generated **2,415** candidates.
    *   **DHP Apriori:** Generated only **58** candidates.
    *   **Impact:** A **97.6% reduction**. DHP virtually eliminated the "Candidate Explosion" problem found in the baseline.

### 2. The Hidden Cost: Hashing Overhead
While DHP reduced the *number* of candidates, the execution time for the `RETAIL` dataset reveals a hidden cost.

*   **Standard Apriori Time:** ~20.5s (at 0.01 support).
*   **DHP Apriori Time:** ~35.8s (constant across all supports).

**Why was DHP slower on Retail?**
The bottleneck shifted from Pass 2 (counting candidates) to **Pass 1 (generating hashes)**.
In Standard Apriori, Pass 1 only counts single items ($N$ operations per transaction). In DHP, Pass 1 must generate and hash **every possible pair** in a transaction. If a transaction has 20 items, DHP performs:
$$ \text{Operations} = \frac{n(n-1)}{2} = \frac{20 \times 19}{2} = 190 \text{ hash operations} $$
For the dense Retail dataset, this massive volume of hashing operations in the first pass outweighed the time saved by having fewer candidates in the second pass.

### 3. Stability and Predictability
A key advantage observed in the DHP results is **predictability**.
*   **Standard Apriori:** Execution time grows exponentially as support decreases (due to candidate explosion).
*   **DHP Apriori:** Execution time remains flat ($\sim 35s$ for Retail) regardless of support level. The hash table cost is fixed, making DHP a more stable choice for systems where memory spikes are unacceptable.

### Summary Comparison

| Feature | Standard Apriori | DHP Apriori |
| :--- | :--- | :--- |
| **Candidate Set ($C_2$)** | High (prone to explosion) | **Extremely Low (Optimized)** |
| **Pass 1 CPU Cost** | Low (Linear scan) | **High (Quadratic Hashing)** |
| **Execution Time Trend** | Sensitive to Min Support | **Stable / Flat** |
| **Best Use Case** | Sparse data, higher support | Dense data, low support, memory constraints |

### Next Steps
We have seen that **Standard Apriori** suffers from candidate explosion, while **DHP** suffers from initial hashing overhead on large datasets.

Phase 3 will implement **Sampling Apriori**, which attempts to solve the speed bottleneck by processing only a random subset of the database, trading theoretical accuracy for significantly improved performance.
